In [6]:
%%time
file_path = r"C:\Users\lin\Videos\Captures\梦幻西游：129五开带魔花果山的另类组合，效率逆天。第（1_2）集_梦幻西游2_游戏资讯 - Google Chrome 2022-09-29 22-14-24.mp4"


class Node:
    name: str
    size: int
    start: int
    children: list

    def __init__(self, name, size, start):
        self.name = name
        self.size = size
        self.start = start
        self.children = []

    def __repr__(self):
        res = []
        res.append(f"name:{self.name}, size:{self.size}, start:{self.start}")
        for child in self.children:
            res.append(f"  {child.__repr__()}")
        return "\n".join(res)


moov = b"moov"
mvhd = b"mvhd"
trak = b"trak"
udta_sub_atom = [b"tagc"]
trak_sub_atom = [
    b"prfl",
    b"tkhd",
    b"tapt",
    b"clip",
    b"matt",
    b"edts",
    b"tref",
    b"txas",
    b"load",
    b"imap",
    b"udta",
    b"mdia",
]
mdia_sub_atom = [b"mdhd", b"elng", b"hdlr", b"minf", b"udta"]

stbl_sub_atom = [b"stsd", b"stts", b'ctts', b"cslg", b'stco', b'stsc', b'stsz']

def if_node(data, d, name, start):
    node = None
    if data[d:d+4] == name:
        node = Node(
            name, int.from_bytes(data[d - 4 : d], byteorder="big"), start + d - 4
        )

    return node

def parse_stts(data):
    size = data[:4]
    type_ = data[4:8]
    version = data[8:9]
    flags = int.from_bytes(data[9:12], byteorder="big")
    num_entries = int.from_bytes(data[12:16], byteorder="big")
    ttst = data[16:]
    sample_count, sample_cost = 0, 0
    samples = []
    count =0
    for i in range(0, len(ttst), 8):
        sample_count = int.from_bytes(ttst[i:i+4], byteorder="big")
        sample_cost = int.from_bytes(ttst[i+4:i+8], byteorder="big")
        count+=sample_count
        samples.append({
            "count": sample_count,
            "cost": sample_cost
        })
    # print(count)
    # print(samples)

def parse_stbl(parent, data, start):
    parent_data = data[start : parent.size+start]
    for d in range(parent.size-4):
        for name in stbl_sub_atom:
            node = if_node(parent_data, d, name, parent.start)
            if node:
                if name == b'stts':
                    parse_stts(parent_data[d-4:d-4+node.size])
                parent.children.append(node)
                continue

def parse_minf(parent, data, start):
    parent_data = data[start : parent.size+start]
    for d in range(parent.size-4):
        node = if_node(parent_data, d, b"stbl", parent.start)
        if node:
            parse_stbl(node, data, start+d-4)
            parent.children.append(node)
            continue


def parse_mdia(parent, data, start):
    parent_data = data[start : parent.size + start]
    for d in range(parent.size-4):
        for name in mdia_sub_atom:
            node = if_node(parent_data, d, name, parent.start)
            if node:
                if node.name == b"minf":
                    parse_minf(node, data, start+d-4)
                mdia_node.children.append(node)
                continue


with open(file_path, "rb") as fp:
    data = fp.read()
    moov_pos = data.find(b'moov')
    fp.seek(7022189 - 4)
    data = fp.read(4)
    moov_size = int.from_bytes(data, byteorder="big")
    fp.seek(7022189 - 4)
    data = fp.read(moov_size)
    moov_atom = Node(moov, moov_size, 7022189 - 4)
    trak_atom = None
    mdia_list = []
    d = 0
    while d < moov_size-4:
        node = if_node(data, d, mvhd, moov_atom.start)
        if node:
            moov_atom.children.append(node)
            d += node.size
            node = None
            continue
        node = if_node(data, d, trak, moov_atom.start)
        if node:
            trak_atom = node
            moov_atom.children.append(node)
            node = None
            d += 1
            continue
        for name in trak_sub_atom:
            node = if_node(data, d, name, moov_atom.start)
            if node:
                if name == b"mdia":
                    mdia_list.append(node)
                trak_atom.children.append(node)
                d += node.size
                node = None
                break
        else:
            d += 1
    for mdia_node in mdia_list:
        relativ_pos = mdia_node.start - moov_atom.start
        parse_mdia(mdia_node, data, relativ_pos)
    # print(udta_data[:1024])
    # print(udta_data.find(b'©nam'))

    # for i in range(udta_node.size):
    #     node = if_node(udta_data, i, b'tagc', udta_node.start)
    #     if node:
    #         print(node)
moov_atom

CPU times: total: 15.6 ms
Wall time: 14 ms


name:b'moov', size:1055059, start:7022185
  name:b'mvhd', size:108, start:7022193
  name:b'trak', size:3834, start:7022301
  name:b'tkhd', size:92, start:7022309
  name:b'mdia', size:3734, start:7022401
  name:b'mdhd', size:32, start:7022409
  name:b'hdlr', size:45, start:7022441
  name:b'minf', size:3649, start:7022486
  name:b'stbl', size:3585, start:7022550
  name:b'stsd', size:149, start:7022558
  name:b'stts', size:1056, start:7022707
  name:b'ctts', size:1016, start:7023763
  name:b'stsc', size:52, start:7024779
  name:b'stsz', size:1120, start:7024831
  name:b'stco', size:128, start:7025951
  name:b'trak', size:2473, start:7026135
  name:b'tkhd', size:92, start:7026143
  name:b'mdia', size:2373, start:7026235
  name:b'mdhd', size:32, start:7026243
  name:b'hdlr', size:45, start:7026275
  name:b'minf', size:2288, start:7026320
  name:b'stbl', size:2228, start:7026380
  name:b'stsd', size:100, start:7026388
  name:b'stts', size:24, start:7026488
  name:b'stsc', size:256, start:702

In [18]:
%%time
root = None
node = None
p = 0
atom_name_ptr = 0

def is_atom(data, atom_type):
    return data[0] == atom_type[0] and data[1] == atom_type[1] and data[2] == atom_type[2] and data[3] == atom_type[3]

def char2int32(data):
    return (data[0] << 24) | (data[1] << 16) | (data[2] << 8) | data[3]
all_atom_names = ["moov", "mvhd", "trak",
    "prfl",
    "tkhd",
    "tapt",
    "clip",
    "matt",
    "edts",
    "tref",
    "txas",
    "load",
    "imap",
    "udta",
    "mdia",
    "mdhd", "elng", "hdlr", "minf", "udta",
    "stsd", "stts", "ctts", "cslg", "stco", "stsc", "stsz"
    ]
all_atom_names = [name.encode() for name in all_atom_names]
with open(file_path, "rb") as fp:
    data = fp.read()
    while p < len(data)-4:
        while atom_name_ptr < len(all_atom_names):
            if data[p:p+4] == all_atom_names[atom_name_ptr]:
                break
            atom_name_ptr+=1
        # if atom_name_ptr < len(all_atom_names):
            # if node:
            #     node.next = Node(atom_name_ptr[0], data[p-4:], char2int32(data[p-4:p]), p-4)
            #     node = node.next
            # if root is None: 
            #     root = Node(atom_name_ptr[0], data[p-4:], char2int32(data[p-4:p]), p-4)
            #     node = root
        p+=1 
print(root)

None
CPU times: total: 1.28 s
Wall time: 1.28 s


In [2]:
from moovparser.moovparser import Node, parse_nodes

In [3]:
%%timeit
file_path = r"C:\Users\lin\Videos\Captures\梦幻西游：129五开带魔花果山的另类组合，效率逆天。第（1_2）集_梦幻西游2_游戏资讯 - Google Chrome 2022-09-29 22-14-24.mp4"
with open(file_path, "rb") as fp:
    data = fp.read()
    # moov_pos = data.find(b'moov')
    node = parse_nodes(data)
    del data
node

5.47 ms ± 123 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
hex(1055059)

In [17]:
%%timeit
0x00<<24|0x10<<16|0x19<<8|0x53

4.62 ns ± 0.0552 ns per loop (mean ± std. dev. of 7 runs, 100,000,000 loops each)


In [19]:
%%timeit
int.from_bytes(b"\x00\x10\x19\x53", byteorder="big")

78.6 ns ± 0.104 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [1]:
file_path = r"C:\Users\lin\Videos\Captures\梦幻西游：129五开带魔花果山的另类组合，效率逆天。第（1_2）集_梦幻西游2_游戏资讯 - Google Chrome 2022-09-29 22-14-24.mp4"
with open(file_path, "rb") as fp:
    fp.seek(263092)
    print(fp.read(193))

b"'\n\x03 \x04\x02\x01\x00\x80@ \x10\x08\x04\x02\x01\x00\x80@ \x10\x08\x04\x02\x01\x00\x80@ \x10\x08\x04\x02\x01\x00\x80@ \x10\x08\x04\x02\x01\x00\x80@ \x10\x08i/\xa4\x00\x00\x00\x00\x00}>\x9fO\xa7\xd3\xe9\xf4\xfa}>\x9fO\xa7\xd3\xe9\xf4\xfa}>\x9fO\xa7\xd3\xe9\xf4\xd7E\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14QE\x14Q\xc0"
